<a href="https://colab.research.google.com/github/KeunhoLee/gan_thumbnail/blob/main/get_thumnail_naverwebtoon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
from bs4 import BeautifulSoup
import requests
import pandas as  pd
import pickle
from urllib.request import urlretrieve
from tqdm import tqdm
import time, random
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive/')

**추출대상 속성**
1. 제목
2. 페이지 링크
  - 2.1 작가명
  - 2.2 장르명
  - 2.3 마지막화 날짜
  - 2.4 첫화 날짜
  - 2.5 작품소개 텍스트
3. 썸네일 링크

### **웹툰 정보 Scraping**

In [36]:
base_url = "https://comic.naver.com"
url = base_url + "/webtoon/creation.nhn"

In [37]:
html = requests.get(url)
soup = BeautifulSoup(html.text, "html.parser")
html.close()

thumb_list = soup.findAll("div", {"class": "thumb"})

In [69]:
result_list = []

for i, t_l in enumerate(thumb_list):

    tmp_dict = { "id" : i,  
      "page_link" : base_url + t_l.findAll("a")[0]["href"],
      "title" : t_l.findAll("a")[0]["title"],
      "thumbnail_link" : t_l.findAll("a")[0].find("img")["src"]
    }

    result_list.append(tmp_dict)

In [70]:
webtoon_info = pd.DataFrame(result_list)
with open('/content/gdrive/MyDrive/saap_pjt_1_thumbnail/webtoon_info.pickle', 'wb') as f:
    pickle.dump(webtoon_info, f, pickle.HIGHEST_PROTOCOL)

In [67]:
# with open('/content/gdrive/MyDrive/saap_pjt_1_thumbnail/webtoon_info.pickle', 'rb') as f:
#     webtoon_info = pickle.load(f)

In [72]:
webtoon_info.head(1)

,id,page_link,title,thumbnail_link
0,0,https://comic.naver.com/webtoon//webtoon/list....,가난을 등에 업은 소녀,https://shared-comic.pstatic.net/thumb/webtoon...


In [78]:
# print('작품 갯수 : ',webtoon_info.shape[0])
print('작품 갯수 : ',len(webtoon_info["title"].unique()))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs
작품 갯수 :  1260


### **썸네일 다운로드**

두번 실행할 필요 없음, IP밴 주의할 것

In [91]:
%time
mean_delay = 0.5 #IP밴을 막기위해 천천히 다운로드 평균 0.5초마다 1개씩 다운로드

print("예상 다운로드 시간 : ", mean_delay*webtoon_info.shape[0], " 초")

for i in tqdm(range(webtoon_info.shape[0])):
  # print(img)

  id = webtoon_info.id[i]
  img_link = webtoon_info.thumbnail_link[i]

  urlretrieve( img_link , "/content/gdrive/MyDrive/saap_pjt_1_thumbnail/thumbnails/naver_id_" + str(id) + ".jpg")

  time.sleep(random.expovariate(1/mean_delay))

  0%|          | 0/1010 [00:00<?, ?it/s]

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs
예상 다운로드 시간 :  630.0  초


100%|██████████| 1010/1010 [20:14<00:00,  1.20s/it]
